<a href="https://colab.research.google.com/github/capitanalegria/MUGSHOTGAN/blob/main/MUGSHOT_GAN_pynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MUGSHOT_GAN set up libraries, modules, mount drive

In [ ]:
#@title Mount google drive, create main folders (mugshotGAN, GoogleImageDL, jojo, Real-ESRGAN, pix2pix, RESULTS). { display-mode: "form" }
from google.colab import drive
drive.mount('/content/drive/')

import os

!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
!pip install tqdm gdown scikit-learn==0.22 scipy lpips dlib opencv-python wandb
!nvidia-smi
!pip install opencv-contrib-python --upgrade

%load_ext autoreload
%autoreload 2

work_folder='/content/drive/MyDrive/'
mugshotGAN_folder=work_folder+'mugshotGAN/'
GoogleImageDL=mugshotGAN_folder+'GoogleImageDL/'
RealESRGAN_folder=mugshotGAN_folder+'realESRGAN/'
JoJo_folder=mugshotGAN_folder+'JoJo/'
pix2pix_folder=mugshotGAN_folder+'pix2pix/'
RESULTS=mugshotGAN_folder+'RESULTS/'


os.makedirs(mugshotGAN_folder, exist_ok=True)
os.makedirs(GoogleImageDL, exist_ok=True)
os.makedirs(JoJo_folder, exist_ok=True)
os.makedirs(RealESRGAN_folder, exist_ok=True)
os.makedirs(pix2pix_folder, exist_ok=True)
os.makedirs(RESULTS, exist_ok=True)

%cd {mugshotGAN_folder}

In [ ]:
#@title Load JoJoGAN from github repository.
!git clone https://github.com/mchong6/JoJoGAN.git {JoJo_folder}
%cd {JoJo_folder}

In [ ]:
#@title Setup JoJoGAN and import essential libraries. { form-width: "100px", display-mode: "form" }
#@markdown This will take a few minutes.
!wget -nc https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip
!sudo unzip ninja-linux.zip -d /usr/local/bin/
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force

%load_ext autoreload
%autoreload 2

from google.colab.patches import cv2_imshow
import cv2
import dlib
import imutils
from imutils import face_utils
from imutils.video import count_frames
from pathlib import Path
from scipy import ndimage
from matplotlib import pyplot as plt
import imageio as io

import torch
torch.backends.cudnn.benchmark = True
from torchvision import transforms, utils
from util import *
from PIL import Image
import math
import random

import numpy as np
from torch import nn, autograd, optim
from torch.nn import functional as F
from tqdm import tqdm
import wandb
from model import *
from e4e_projection import projection as e4e_projection

from google.colab import files
from copy import deepcopy
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

os.makedirs('inversion_codes', exist_ok=True)
os.makedirs('style_images', exist_ok=True)
os.makedirs('style_images_aligned', exist_ok=True)
os.makedirs('models', exist_ok=True)

In [ ]:
#@title Create Folder Maker function, loop thru folder function as List Loader, and load MODULES folder.
#folder function===================================================
def foldermaker(folder_name=''):
        if os.path.exists(folder_name):
            print('folder {}'.format(folder_name)+' READY!')
        else:
            os.mkdir('{}'.format(folder_name))
            print('folder {}'.format(folder_name)+' READY!')

#load contents of a folder and pass them during looping functions
def list_loader(folder):
    folder_array=os.listdir(folder)
    total_files=len(folder_array)
    return (folder_array, total_files)

#main folders ====================================================
MODELS='{}'.format(mugshotGAN_folder)+'MODELS/'
foldermaker(MODELS)
JoJoMODELS=JoJo_folder+'models/'
foldermaker(JoJoMODELS)

#import modules =================================================
modules=mugshotGAN_folder+'MODULES/'
foldermaker(modules)

In [ ]:
#@title Set up Google Image DL.
#@markdown 
#@markdown

##@markdown What image do you want to download:
##@markdown

#search_image = 'Mugshot' #@param {type: 'string'}
#search_image=search_image.replace(' ', '_') 
#image_folder='{}'.format(RESULTS)+str(search_image)+'/'


#enter face you want ============================================
#search_image=input('what image do you want to search? ')
%cd {work_folder}
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver

#install google image download
!git clone https://github.com/Joeclinton1/google-images-download.git {GoogleImageDL}
%cd {GoogleImageDL} 
!ls
!sudo python setup.py install

#torch.save(search_image, f'{mugshotGAN_folder}search_image.torch', _use_new_zipfile_serialization=False)
##@markdown
##@markdown Download images or set variable?
#download_images = "Download" #@param ["Download", "Set variable"]


# SKIP!!!! if you want to generate digital MUGSHOTS  
#================================
OR Download images and train your OWN pix2pix BY RUNNING THIS SECTION.


(skip this step if you are using the pre-trained checkpoint mugshotGAN_4, included below from mint.tokyo)

In [ ]:
#@title Load search image variable.
#@markdown 
#@markdown What image do you want to download:
#@markdown 
search_image = 'Mugshot' #@param {type: 'string'}
search_image=search_image.replace(' ', '_') 

#search_image=torch.load(f'{mugshotGAN_folder}search_image.torch')
image_folder='{}'.format(RESULTS)+str(search_image)+'/'

In [ ]:
#@title Download images.
!googleimagesdownload -k {search_image} --chromedriver /usr/bin/chromedriver -ri -l 2000 -o {RESULTS} -s large -f jpg

%cd {mugshotGAN_folder}

In [ ]:
#@title Download and/or load JoJoGAN essential models
#@markdown You may optionally enable downloads with pydrive in order to authenticate and avoid drive download limits. DOWNLOAD WITH PYDRIVE ON YOUR FIRST RUN.
#
#@markdown Defaults to downloaded models. SMASH THAT GO BUTTON!
%cd {JoJo_folder}

download_with_pydrive = True #@param {type:"boolean"}    
device = 'cuda' #@param ['cuda', 'cpu']

!wget -nc http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2
!mv shape_predictor_68_face_landmarks.dat models/dlibshape_predictor_68_face_landmarks.dat
#!cp {JoJoMODELS}shape_predictor_68_face_landmarks.dat models/dlibshape_predictor_68_face_landmarks.dat
#!cp {JoJoMODELS}stylegan2-ffhq-config-f.pt models/stylegan2-ffhq-config-f.pt
#!cp {JoJoMODELS}e4e_ffhq_encode.pt models/e4e_ffhq_encode.pt
%matplotlib inline


drive_ids = {
    "stylegan2-ffhq-config-f.pt": "1Yr7KuD959btpmcKGAUsbAk5rPjX2MytK",
    "e4e_ffhq_encode.pt": "1o6ijA3PkcewZvwJJ73dJ0fxhndn0nnh7",
    "restyle_psp_ffhq_encode.pt": "1nbxCIVw9H3YnQsoIPykNEFwWJnHVHlVd",
    "arcane_caitlyn.pt": "1gOsDTiTPcENiFOrhmkkxJcTURykW1dRc",
    "arcane_caitlyn_preserve_color.pt": "1cUTyjU-q98P75a8THCaO545RTwpVV-aH",
    "arcane_jinx_preserve_color.pt": "1jElwHxaYPod5Itdy18izJk49K1nl4ney",
    "arcane_jinx.pt": "1quQ8vPjYpUiXM4k1_KIwP4EccOefPpG_",
    "arcane_multi_preserve_color.pt": "1enJgrC08NpWpx2XGBmLt1laimjpGCyfl",
    "arcane_multi.pt": "15V9s09sgaw-zhKp116VHigf5FowAy43f",
    "sketch_multi.pt": "1GdaeHGBGjBAFsWipTL0y-ssUiAqk8AxD",
    "disney.pt": "1zbE2upakFUAx8ximYnLofFwfT8MilqJA",
    "disney_preserve_color.pt": "1Bnh02DjfvN_Wm8c4JdOiNV4q9J7Z_tsi",
    "jojo.pt": "13cR2xjIBj8Ga5jMO7gtxzIJj2PDsBYK4",
    "jojo_preserve_color.pt": "1ZRwYLRytCEKi__eT2Zxv1IlV6BGVQ_K2",
    "jojo_yasuho.pt": "1grZT3Gz1DLzFoJchAmoj3LoM9ew9ROX_",
    "jojo_yasuho_preserve_color.pt": "1SKBu1h0iRNyeKBnya_3BBmLr4pkPeg_L",
    "art.pt": "1a0QDEHwXQ6hE_FcYEyNMuv5r5UnRQLKT",
}

# from StyelGAN-NADA
class Downloader(object):
    def __init__(self, use_pydrive):
        self.use_pydrive = use_pydrive

        if self.use_pydrive:
            self.authenticate()
        
    def authenticate(self):
        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        self.drive = GoogleDrive(gauth)
    
    def download_file(self, file_name):
        file_dst = os.path.join('models', file_name)
        file_id = drive_ids[file_name]
        if not os.path.exists(file_dst):
            print(f'Downloading {file_name}')
            if self.use_pydrive:
                downloaded = self.drive.CreateFile({'id':file_id})
                downloaded.FetchMetadata(fetch_all=True)
                downloaded.GetContentFile(file_dst)
            else:
                !gdown --id $file_id -O $file_dst

if download_with_pydrive == True:
    downloader = Downloader(download_with_pydrive)
    downloader.download_file('stylegan2-ffhq-config-f.pt')
    downloader.download_file('e4e_ffhq_encode.pt')

latent_dim = 512

# Load original generator
original_generator = Generator(1024, latent_dim, 8, 2).to(device)
ckpt = torch.load('models/stylegan2-ffhq-config-f.pt', map_location=lambda storage, loc: storage)
original_generator.load_state_dict(ckpt["g_ema"], strict=False)
mean_latent = original_generator.mean_latent(10000)

# to be finetuned generator
generator = deepcopy(original_generator)


transform = transforms.Compose(
    [
        transforms.Resize((1024, 1024)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

In [ ]:
#@title Run JoJo face detect, resize and crop faces.
#@markdown Choose what kind of output for the JoJoGAN face detect you want:
jojo_facedetect_output = "jpgs_and_variables" #@param ["jpgs_and_variables", "only_variables"]

#@markdown This cell autosaves a pytorch checkpoint var with the results to reuse in case of crash.

def jojo_cuda_DetectResize(folder=image_folder, search_image=search_image):

#cuda as device ===================================================================
    device = 'cuda'

 #result folder n variable ========================================================   
    FACES=RESULTS+search_image+'_FACES/'
    foldermaker(FACES)
    aligned_faces=[]
    my_ws=[]

#load folder======================================================================
    filepath, total_files=list_loader(folder)

#loop thru folder=================================================================
    for f in range (0,total_files):
        img=folder+filepath[f]
        #print(img)

#skip folders n weird formats ===================================================
        img_extensions=('.jpg', '.png')
        if img.endswith(img_extensions):
            try:
                plt.rcParams['figure.dpi'] = 150

# uploaded = files.upload()
# filepath = list(uploaded.keys())[0]
                name = strip_path_extension(img)+'.pt'

# aligns and crops face =====================
                aligned_face= align_face(img)
                aligned_face.save(FACES+'{}_JOJOface_{}.jpg'.format(search_image, f), "JPEG", quality=100, optimize=True, progressive=True)
                my_w = e4e_projection(aligned_face, name, device).unsqueeze(0)
                #print(aligned_face)
                #display_image(aligned_face, title='Aligned face')
                aligned_faces.append(aligned_face)
                my_ws.append(my_w)
                f+=1
            except Exception:
                pass
    return (FACES, device, aligned_faces, my_ws)


def jojo_cuda_DetectResize_nosave(folder=image_folder):

#cuda as device ===================================================================
    device = 'cuda'

 #result folder n variable ========================================================   
    FACES=RESULTS+search_image+'_FACES/'
    foldermaker(FACES)
    aligned_faces=[]
    my_ws=[]

#load folder======================================================================
    filepath, total_files=list_loader(folder)

#loop thru folder=================================================================
    for f in range (0,total_files):
        img=folder+filepath[f]
        #print(img)

#skip folders n weird formats ===================================================
        img_extensions=('.jpg', '.png')
        if img.endswith(img_extensions):
            try:
                plt.rcParams['figure.dpi'] = 150

# uploaded = files.upload()
# filepath = list(uploaded.keys())[0]
                name = strip_path_extension(img)+'.pt'

# aligns and crops face =====================
                aligned_face= align_face(img)
                #aligned_face.save(FACES+'{}_JOJOface_{}.jpg'.format(search_image, f), "JPEG", quality=100, optimize=True, progressive=True)

                # my_w = restyle_projection(aligned_face, name, device, n_iters=1).unsqueeze(0)
                my_w = e4e_projection(aligned_face, name, device).unsqueeze(0)

                #print(aligned_face)
                #display_image(aligned_face, title='Aligned face')
                my_ws.append(my_w)
                aligned_faces.append(aligned_face)
                f+=1
            except Exception:
                pass
    return (FACES, device, aligned_faces, my_ws)
run_after_pix2pix = False #@param {type:"boolean"}
if run_after_pix2pix == True:
    image_folder_nu=RealESRGAN_result_folder
if jojo_facedetect_output == "jpgs_and_variables":
    FACES, device, aligned_faces, my_ws=jojo_cuda_DetectResize(image_folder, search_image)
    torch.save(aligned_faces, f'{mugshotGAN_folder}{search_image}_aligned_faces.torch', _use_new_zipfile_serialization=False)
    torch.save(my_ws, f'{JoJo_folder}{search_image}_my_ws.torch', _use_new_zipfile_serialization=False)
else:
    FACES, device, aligned_faces, my_ws=jojo_cuda_DetectResize_nosave(image_folder)
    torch.save(aligned_faces, f'{mugshotGAN_folder}{search_image}_aligned_faces.torch', _use_new_zipfile_serialization=False)
    torch.save(my_ws, f'{JoJo_folder}{search_image}_my_ws.torch', _use_new_zipfile_serialization=False)

In [ ]:
#@title Run this cell to reload pytorch checkpoints for jojo face variables in case of crash.
#@markdown You can skip if this is first run.
my_ws=torch.load(f'{mugshotGAN_folder}{search_image}_my_ws.torch')
aligned_faces=torch.load(f'{mugshotGAN_folder}{search_image}_aligned_faces.torch')
FACES=RESULTS+'{}_FACES/'.format(search_image)

In [ ]:
#@title Optionally transfer all images from multiple searches into one folder. { display-mode: "form" }
if search_image_nu != None:
    mega_image_folder=RESULTS+'{}_mega_folder/'.format(search_image_nu)
else:
    mega_image_folder=RESULTS+'{}_mega_folder/'.format(search_image)
foldermaker(mega_image_folder)
#@markdown Enter folder to copy.
face_folder='/content/drive/MyDrive/mugshotGAN/pix2pix/results/mugshotGAN_4/test_latest/images'#@param
!rsync -a {face_folder} {mega_image_folder}

In [ ]:
#@title Set up Real-ESRGAN to upscale the image DATASET.
# Clone Real-ESRGAN and enter the Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git {RealESRGAN_folder}
%cd {RealESRGAN_folder}
# Set up the environment
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop

# Download the pre-trained model
!wget -nc https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models

import shutil

os.makedirs('temp_inputs', exist_ok=True) 
FACES=RESULTS+'{}_FACES/'.format(search_image)
RealESRGAN_load_folder = FACES
RealESERGAN_upload_folder=RealESRGAN_folder+'temp_inputs/'
#!rsync -a {RealESRGAN_load_folder} {RealESRGAN_upload_folder}
if search_image_nu != None:
    fake_folder=RESULTS+search_image_nu+'_FAKE/'
    RealESRGAN_result_folder = RESULTS+search_image_nu+'_ESRGAN/'    
else:
    RealESRGAN_result_folder = RESULTS+search_image+'_ESRGAN/'
foldermaker(RealESRGAN_result_folder)
realERSGAN_input = fake_folder #@param ["mega_image_folder", "FACES", "image_folder", "None", "face_folder", "fake_folder"] {type:"raw"}
!python inference_realesrgan.py -n RealESRGAN_x4plus --input {realERSGAN_input} --outscale 3.5 --face_enhance --output {RealESRGAN_result_folder}

In [ ]:
#@title Create train data: { form-width: "200px", display-mode: "form" }
#@markdown Define Face Crop function and crop faces. Resize and split into folders for pix2pix GAN.

#!cp {modules}FACE_CROPPER.py .
#from FACE_CROPPER import FACE_CROPPER
work_folder='/content/drive/MyDrive/'

#RealESRGAN_result_folder=RESULTS+search_image+'_ESRGAN/'

FACES=RESULTS+search_image+'_FACES/'
RealESRGAN_result_folder=FACES

#@markdown Select size options:
width_and_height = 1024 #@param ["256", "512", "1024"] {type:"raw"}

import cv2
import dlib
import sys
import os

def FACE_CROPPER (folder, width=width_and_height, height=width_and_height):
    croppedFACES=RESULTS+'{}_crop/'.format(search_image)
    foldermaker(croppedFACES)

    A_folder=croppedFACES+'A/'
    B_folder=croppedFACES+'B/'
    foldermaker(A_folder)
    foldermaker(B_folder)

    A_train=A_folder+'train/'
    B_train=B_folder+'train/'
    foldermaker(A_train)
    foldermaker(B_train)

    A_val=A_folder+'val/'
    B_val=B_folder+'val/'
    foldermaker(A_val)
    foldermaker(B_val)
    
    A_test=A_folder+'test/'
    B_test=B_folder+'test/'
    foldermaker(A_test)
    foldermaker(B_test)
    
    dim=(width, height)

#model ================================================================
    detector=dlib.get_frontal_face_detector()
    model=dlib.shape_predictor(JoJo_folder+'models/dlibshape_predictor_68_face_landmarks.dat')

#source=================================================================
    FACES, frameCOUNT=list_loader(folder)

#loop trhu folder ======================================================
    for f in range(0,frameCOUNT):
        print(f)

#string path ===========================================================
        imash=folder+FACES[f]
        img_extensions=('.jpg','.png')

#only JPGS AND PNGS  ===================================================
        if imash.endswith(img_extensions):
            try:

#DLIB LOAD==============================================================
                IMG=dlib.load_rgb_image(imash)   
#CV LOAD  ==============================================================
                #IMG = cv2.imread(imash)[:,:,::-1]
            
#find face =============================================================
                FACE=detector(IMG, 1)
            
#Pass2landmark =========================================================
                landmark_array=[]
                for k, d, in enumerate(FACE):
                    landmarks=model(IMG, d)

#select only landmarks 0-27 to cutout face=============================
                    for n in range(0,27):
                        x=landmarks.part(n).x
                        y=landmarks.part(n).y
                        landmark_array.append((x, y))
                        #cv2.circle(IMG, (x,y), 2, (255, 255, 0), -1)
            
 #detect if face is frontal ("manually")
                frontson1=landmark_array[0][1] 
                frontson2=landmark_array[17][1]

                cutout_guide=[]

#outline path from landmarks 0-27 ===================================
                for i in range (15, -1, -1):
                    from_coordinate=landmark_array[i+1]
                    to_coordinate=landmark_array[i]
                    cutout_guide.append(from_coordinate)
            
                from_coordinate=landmark_array[0]
                to_coordinate=landmark_array[17]
                cutout_guide.append(from_coordinate)

                for i in range (17, 20):
                    from_coordinate=landmark_array[i]
                    to_coordinate=landmark_array[i+1]
                    cutout_guide.append(from_coordinate)
            
                from_coordinate=landmark_array[19]
                to_coordinate=landmark_array[24]
                cutout_guide.append(from_coordinate)

                for i in range (24, 26):
                    from_coordinate=landmark_array[i]
                    to_coordinate=landmark_array[i+1]
                    cutout_guide.append(from_coordinate)
                
                from_coordinate=landmark_array[26]
                to_coordinate=landmark_array[16]
                cutout_guide.append(from_coordinate)
                cutout_guide.append(to_coordinate)

#outline face===============================================================
                for i in range(0, len(cutout_guide)-1):
                    from_coordinates=cutout_guide[i]
                    to_coordinates=cutout_guide[i+1]
                    IMG2=cv2.line(IMG, from_coordinate, to_coordinate, (255,255,255), 1)
            
#crop it    ================================================================
                mask=np.zeros((IMG2.shape[0], IMG2.shape[1]))
                mask=cv2.fillConvexPoly(mask, np.array(cutout_guide), 1)
                mask=mask.astype(np.bool_)
                cropped=np.zeros_like(IMG2)
                cropped[mask]=IMG2[mask]

#convert to rgb to avoid blue image from BGR ===============================
                IMG=cv2.cvtColor(IMG, cv2.COLOR_BGR2RGB)
                cropped=cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)
                
#jpg cropped  ==============================================================
                IMG=cv2.resize(IMG, dim, interpolation = cv2.INTER_AREA)
                cropped=cv2.resize(cropped, dim, interpolation = cv2.INTER_AREA)
                data_division=int(frameCOUNT*0.9)
                if f <= data_division:
                    cv2.imwrite(A_train+'A{}.jpg'.format(f), IMG)
                    cv2.imwrite(B_train+'B{}.jpg'.format(f), cropped)
                elif f >= data_division and f <= (int(data_division*1.07)):
                    cv2.imwrite(A_val+'A{}.jpg'.format(f), IMG)
                    cv2.imwrite(B_val+'B{}.jpg'.format(f), cropped)
                else:
                    cv2.imwrite(A_test+'A{}.jpg'.format(f), IMG)
                    cv2.imwrite(B_test+'B{}.jpg'.format(f), cropped)
                f+=1
            except Exception:
                pass
    return croppedFACES, A_folder, B_folder


croppedFACES, A_folder, B_folder=FACE_CROPPER(RealESRGAN_result_folder)

In [ ]:
#@title load pix2pix 
#@markdown optionally download git if its not already in work_folder:
download_pix2pix = True #@param {type:"boolean"}
if download_pix2pix==True:
    !git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix {pix2pix_folder}
%cd {pix2pix_folder}
!pip install -r requirements.txt

In [ ]:
#@title Format DATA for train pix2pix GAN.
#@markdown (A, B, and AB with corresponding "test," "train" and "val" folder.)
#croppedFACES=RESULTS+'{}_crop/'.format(search_image)

#A_folder=croppedFACES+'A/'
#B_folder=croppedFACES+'B/'
AB_folder=croppedFACES+'AB/'
foldermaker(AB_folder)

def combine_AB(A, B, path_AB):
    Aa, frameCOUNT=list_loader(A)
    Bb, frameCOUNT=list_loader(B)
    foldermaker(path_AB)
    print(frameCOUNT)

    for f in range (0, frameCOUNT):
        print(f)
        path_A=A+Aa[f]
        path_B=B+Bb[f]
    
        im_A = cv2.imread(path_A) # python2: cv2.CV_LOAD_IMAGE_COLOR; python3: cv2.IMREAD_COLOR
        im_B = cv2.imread(path_B) # python2: cv2.CV_LOAD_IMAGE_COLOR; python3: cv2.IMREAD_COLOR
        im_AB = np.concatenate([im_A, im_B],1)

        cv2.imwrite(path_AB+'{}_AB{}.jpg'.format(search_image,f), im_AB)
        f+=1

combine_AB(A_folder+'train/', B_folder+'train/', AB_folder+'train/')
combine_AB(A_folder+'test/', B_folder+'test/', AB_folder+'test/')
combine_AB(A_folder+'val/', B_folder+'val/', AB_folder+'val/')

#!python datasets/combine_A_and_B.py --fold_A {A_folder} --fold_B {B_folder} 

In [ ]:
#@title Train pix2pix model with formatted Data. { display-mode: "form" }
#@markdown Name checkpoint to reuse when generating images.
%cd {pix2pix_folder}
model_checkpoint = "mugshotGAN_4" #@param {type:"string"}
croppedFACES_ab=RESULTS+search_image+'_crop/'+'AB/'
#@markdown Select direction of generation in GAN and total epochs:
model_direction = "BtoA" #@param ["AtoB", "BtoA"]
num_epochs = "4000" #@param {type:"string"}
#@markdown To continue training check:
continue_training = True #@param {type:"boolean"}
if model_checkpoint == "mugshotGAN_4" and continue_training ==True:
    %cd {pix2pix_folder+'checkpoints/'}
    !wget -nc https://mint.tokyo/MUGSHOT_GAN/mugshotGAN_4.zip
    !sudo unzip mugshotGAN_4.zip
    %cd 
    %cd {pix2pix_folder}
if continue_training == True:
    restart_epoch = "2150" #@param {type:"string"}
    continue_params='--continue_train --epoch_count {}'.format(restart_epoch)
else:
    continue_params=''
#--epoch_count 333 --continue_train --n_epochs 900 --n_epochs_decay 900 --ndf 124 --ngf 124 --load_size 572 --crop_size 512
!python train.py --dataroot {croppedFACES_ab} --name {model_checkpoint} --model pix2pix --direction {model_direction} --n_epochs {num_epochs} {continue_params} 


# Get NU IMAGES to Mugshot-ify.

In [ ]:
#@title Set target face variable.
#@markdown 
#@markdown
#@markdown Whose face do you want to download:
#@markdown 
search_image_nu = 'capi' #@param {type: 'string'}
search_image_nu=search_image_nu.replace(' ', '_') 
image_folder_nu=RESULTS+search_image_nu+'/'

torch.save(search_image_nu, f'{mugshotGAN_folder}search_image_nu.torch', _use_new_zipfile_serialization=False)

In [ ]:
#@title Choose DOWNLOAD or UPLOAD images. { form-width: "200px", display-mode: "form" }
face_type = "upload" #@param ["download", "upload"] {allow-input: true}
how_many_dl_imgs = "20000" #@param ["100", "200", "500", "2000"] {allow-input: true}
#@markdown If you chose upload, use one of the following or LEAVE BLANK to UPLOAD from browser.

video_boi = True #@param {type:"boolean"}
zip_boi = False #@param {type:"boolean"}
path_boi = "/content/drive/MyDrive/mugshotGAN/RESULTS/capi_VIDS/" #@param {type:"string"}

if face_type == 'download':
    %cd {mugshotGAN_folder}
    !googleimagesdownload -k {search_image_nu} --chromedriver /usr/bin/chromedriver -ri -l {how_many_dl_imgs} -o {RESULTS} -s large
elif face_type == 'upload' and video_boi == False and zip_boi ==False and path_boi=="":
    %cd 
    %cd {work_folder}
    from google.colab import files
    uploaded = files.upload()
    for fn in uploaded.keys():
        print('User uploaded file "{name}" with length {length} bytes'.format(
            name=fn, length=len(uploaded[fn])))
    image_folder_nu='/content/drive/MyDrive/'

elif face_type == 'upload' and video_boi == False and zip_boi ==False:
    %cd
    %cd {work_folder}
    image_folder_nu=path_boi

elif face_type == 'upload' and video_boi == True and path_boi!='':
    %cd
    %cd {RESULTS+'pathboi_CAPI'}
    def vid2frames(folder):
        foldermaker(image_folder_nu)
        vid_name, total_videos=list_loader(folder) 
    #load video ===================================================
        for f in range(0,total_videos):
            video=folder+vid_name[f]
            vidcap=cv2.VideoCapture(f'{video}')
            success, video=vidcap.read()
            cv2_imshow(video)
            cv2.waitKey(1)
            cv2.destroyAllWindows()

#get number of frames in vid =================================
            #total_frames=count_frames(video)
            count=0
            #print(total_frames)
            print(len(video))
#vertical 2 horizontal vid====================================
            vid_rot=input('rotate?(y,n):')
            while success:

#rotate first ===============================================
                if vid_rot == 'y':
                    ROT_angle={'90': cv2.ROTATE_90_CLOCKWISE,
                    '-90': cv2.ROTATE_90_COUNTERCLOCKWISE, 
                    '180': cv2.ROTATE_180}
                    angle=input('rotate angle?:90,-90,180:')
                    rota=ROT_angle.get(angle)
                    video=cv2.rotate(video, rota)
#save as jpg ================================================
                    cv2.imwrite(image_folder_nu+'{}_vid{}_f{}.jpg'.format(search_image_nu, f, count), video)
                    success,video=vidcap.read()
                    print('Read a new frame: ', success)
                    count+=1
                    f += 1
                    print("EUREEEEEEEKA")
                else:
                    cv2.imwrite(image_folder_nu+'{}_vid{}_f{}.jpg'.format(search_image_nu, f, count), video)
                    success,video=vidcap.read()
                    print('Read a new frame: ', success)
                    count+=1
                    f += 1
                    print("EUREEEEEEEKA")
#vid function =======================================================    
    vid2frames(path_boi)
else:
    from google.colab import files
#video boi=====================================================
    if video_boi == True:
        %cd
        %cd {work_folder}
        vid_folder=RESULTS+'{}_VIDS/'.format(search_image_nu)
        foldermaker(vid_folder)
        image_folder_nu=vid_folder
        %cd {vid_folder}
        vid_name=[]
        uploaded = files.upload()
        for fn in uploaded.keys():
            print('User uploaded file "{name}" with length {length} bytes'.format(        name=fn, length=len(uploaded[fn])))
            vid_name.append(fn)
        from google.colab.patches import cv2_imshow
        def vid2frames(uploaded):
    #load vid path=================================================
            if uploaded== None:
                uploaded=input('video folder:')
            foldermaker(image_folder_nu)
            total_videos=len(uploaded) 
    #load video ===================================================
            for f in range(0,total_videos):
                video=vid_folder+vid_name[f]
                vidcap=cv2.VideoCapture(f'{video}')
                success, video=vidcap.read()
                cv2_imshow(video)
                cv2.waitKey(0)
                
                input 
                cv2.destroyAllWindows()
                
    #get number of frames in vid =================================
                #total_frames=count_frames(video)
                count=0
                #print(total_frames)
                #print(len(video))
    #vertical 2 horizontal vid====================================
                while success:

    #rotate first ===============================================
                    if vid_rot == 'y':
                        ROT_angle={'90': cv2.ROTATE_90_CLOCKWISE,
                        '-90': cv2.ROTATE_90_COUNTERCLOCKWISE, 
                        '180': cv2.ROTATE_180}
                        angle=input('rotate angle?:90,-90,180:')
                        rota=ROT_angle.get(angle)
                        video=cv2.rotate(video, rota)
    #save as jpg ================================================
                        cv2.imwrite(image_folder_nu+'{}_vid{}_f{}.jpg'.format(search_image_nu, f, count), video)
                        success,video=vidcap.read()
                        print('Read a new frame: ', success)
                        count+=1
                        f += 1
                        print("EUREEEEEEEKA")
                    else:
                        cv2.imwrite(image_folder_nu+'{}_vid{}_f{}.jpg'.format(search_image_nu, f, count), video)
                        success,video=vidcap.read()
                        print('Read a new frame: ', success)
                        count+=1
                        f += 1
                        print("EUREEEEEEEKA")
#vid function =======================================================    
        vid2frames(uploaded)
#zip boi=============================================================
    if zip_boi == True:
        %cd
        %cd {RESULTS}
        uploaded = files.upload()
        zip_name=[]
        for fn in uploaded.keys():
            print('User uploaded file "{name}" with length {length} bytes'.format(
                name=fn, length=len(uploaded[fn])))
            zip_name.append(fn)
        import zipfile
        import shutil
        local_path = image_folder_nu
        makefolder(local_path)
        for f in  range (0, len(uploaded)):
            file_name = zip_name[f]#zip_folder #path + dataset
            with zipfile.ZipFile(file_name, 'r') as zip:
                #zip.printdir()
                print('Extracting all the files now...') 
                zip.extractall(local_path) 
                print('Done!')
            f+=1
 

# Test pix2pix on NU IMAGES.

In [ ]:
#@title Download and/or load JoJoGAN models. { form-width: "100px" }
#@markdown RUN THIS STEP and download with pydrive during FIST RUN. You may skip pydrive download if models already in folder.
#
#@markdown Defaults to downloaded models. SMASH THAT GO BUTTON!
%cd {JoJo_folder}

download_with_pydrive = False #@param {type:"boolean"}    
device = 'cuda' #@param ['cuda', 'cpu']

!wget -nc http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2
!mv shape_predictor_68_face_landmarks.dat models/dlibshape_predictor_68_face_landmarks.dat
#!cp {JoJoMODELS}shape_predictor_68_face_landmarks.dat models/dlibshape_predictor_68_face_landmarks.dat
#!cp {JoJoMODELS}stylegan2-ffhq-config-f.pt models/stylegan2-ffhq-config-f.pt
#!cp {JoJoMODELS}e4e_ffhq_encode.pt models/e4e_ffhq_encode.pt
%matplotlib inline


drive_ids = {
    "stylegan2-ffhq-config-f.pt": "1Yr7KuD959btpmcKGAUsbAk5rPjX2MytK",
    "e4e_ffhq_encode.pt": "1o6ijA3PkcewZvwJJ73dJ0fxhndn0nnh7",
    "restyle_psp_ffhq_encode.pt": "1nbxCIVw9H3YnQsoIPykNEFwWJnHVHlVd",
    "arcane_caitlyn.pt": "1gOsDTiTPcENiFOrhmkkxJcTURykW1dRc",
    "arcane_caitlyn_preserve_color.pt": "1cUTyjU-q98P75a8THCaO545RTwpVV-aH",
    "arcane_jinx_preserve_color.pt": "1jElwHxaYPod5Itdy18izJk49K1nl4ney",
    "arcane_jinx.pt": "1quQ8vPjYpUiXM4k1_KIwP4EccOefPpG_",
    "arcane_multi_preserve_color.pt": "1enJgrC08NpWpx2XGBmLt1laimjpGCyfl",
    "arcane_multi.pt": "15V9s09sgaw-zhKp116VHigf5FowAy43f",
    "sketch_multi.pt": "1GdaeHGBGjBAFsWipTL0y-ssUiAqk8AxD",
    "disney.pt": "1zbE2upakFUAx8ximYnLofFwfT8MilqJA",
    "disney_preserve_color.pt": "1Bnh02DjfvN_Wm8c4JdOiNV4q9J7Z_tsi",
    "jojo.pt": "13cR2xjIBj8Ga5jMO7gtxzIJj2PDsBYK4",
    "jojo_preserve_color.pt": "1ZRwYLRytCEKi__eT2Zxv1IlV6BGVQ_K2",
    "jojo_yasuho.pt": "1grZT3Gz1DLzFoJchAmoj3LoM9ew9ROX_",
    "jojo_yasuho_preserve_color.pt": "1SKBu1h0iRNyeKBnya_3BBmLr4pkPeg_L",
    "art.pt": "1a0QDEHwXQ6hE_FcYEyNMuv5r5UnRQLKT",
}

# from StyelGAN-NADA
class Downloader(object):
    def __init__(self, use_pydrive):
        self.use_pydrive = use_pydrive

        if self.use_pydrive:
            self.authenticate()
        
    def authenticate(self):
        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        self.drive = GoogleDrive(gauth)
    
    def download_file(self, file_name):
        file_dst = os.path.join('models', file_name)
        file_id = drive_ids[file_name]
        if not os.path.exists(file_dst):
            print(f'Downloading {file_name}')
            if self.use_pydrive:
                downloaded = self.drive.CreateFile({'id':file_id})
                downloaded.FetchMetadata(fetch_all=True)
                downloaded.GetContentFile(file_dst)
            else:
                !gdown --id $file_id -O $file_dst


if download_with_pydrive == True:
    downloader = Downloader(download_with_pydrive)
    downloader.download_file('stylegan2-ffhq-config-f.pt')
    downloader.download_file('e4e_ffhq_encode.pt')

latent_dim = 512

# Load original generator
original_generator = Generator(1024, latent_dim, 8, 2).to(device)
ckpt = torch.load('models/stylegan2-ffhq-config-f.pt', map_location=lambda storage, loc: storage)
original_generator.load_state_dict(ckpt["g_ema"], strict=False)
mean_latent = original_generator.mean_latent(10000)

# to be finetuned generator
generator = deepcopy(original_generator)


transform = transforms.Compose(
    [
        transforms.Resize((1024, 1024)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

In [ ]:
#@title Run JoJo face detect, resize and crop TARGET faces. { form-width: "200px" }

##@markdown Choose OUTPUT type:
#jojo_facedetect_output = "jpgs_and_variables" #@param ["jpgs_and_variables", "only_variables"]
#@markdown Choose processing device type:
device_type = "cuda" #@param ["cuda", "cpu"]
def jojo_cuda_DetectResize(folder=image_folder_nu, search_image=search_image_nu):

#cuda as device ===================================================================
    device = device_type

 #result folder n variable ========================================================   
    FACES=RESULTS+search_image+'_FACES/'
    foldermaker(FACES)
    aligned_faces=[]
    my_ws=[]

#load folder======================================================================
    filepath, total_files=list_loader(folder)

#loop thru folder=================================================================
    for f in range (0,total_files):
        img=folder+filepath[f]
        #print(img)

#skip folders n weird formats ===================================================
        img_extensions=('.jpg', '.png')
        if img.endswith(img_extensions):
            try:
                plt.rcParams['figure.dpi'] = 150

# uploaded = files.upload()
# filepath = list(uploaded.keys())[0]
                name = strip_path_extension(img)+'.pt'

# aligns and crops face =====================
                aligned_face= align_face(img)
                aligned_face.save(FACES+'{}_JOJOface_{}.jpg'.format(search_image, f), "JPEG", quality=100, optimize=True, progressive=True)
                my_w = e4e_projection(aligned_face, name, device).unsqueeze(0)
                #print(aligned_face)
                #display_image(aligned_face, title='Aligned face')
                aligned_faces.append(aligned_face)
                my_ws.append(my_w)
                f+=1
            except Exception:
                pass
    return (FACES, device, aligned_faces, my_ws)


def jojo_cuda_DetectResize_nosave(folder=image_folder_nu):

#cuda as device ===================================================================
    device = device_type

 #result folder n variable ========================================================   
    FACES=RESULTS+search_image+'_FACES/'
    foldermaker(FACES)
    aligned_faces=[]
    my_ws=[]

#load folder======================================================================
    filepath, total_files=list_loader(folder)

#loop thru folder=================================================================
    for f in range (0,total_files):
        img=folder+filepath[f]
        #print(img)

#skip folders n weird formats ===================================================
        img_extensions=('.jpg', '.png')
        if img.endswith(img_extensions):
            try:
                plt.rcParams['figure.dpi'] = 150

# uploaded = files.upload()
# filepath = list(uploaded.keys())[0]
                name = strip_path_extension(img)+'.pt'

# aligns and crops face =====================
                aligned_face= align_face(img)
                #aligned_face.save(FACES+'{}_JOJOface_{}.jpg'.format(search_image, f), "JPEG", quality=100, optimize=True, progressive=True)

                # my_w = restyle_projection(aligned_face, name, device, n_iters=1).unsqueeze(0)
                my_w = e4e_projection(aligned_face, name, device).unsqueeze(0)

                #print(aligned_face)
                #display_image(aligned_face, title='Aligned face')
                my_ws.append(my_w)
                aligned_faces.append(aligned_face)
                f+=1
            except Exception:
                pass
    return (FACES, device, aligned_faces, my_ws)

#@markdown *This cell autosaves a pytorch checkpoint VARS to resume in case of crash.

FACES, device, aligned_faces, my_ws=jojo_cuda_DetectResize(image_folder_nu, search_image_nu)
torch.save(aligned_faces, f'{mugshotGAN_folder}{search_image_nu}_aligned_faces.torch', _use_new_zipfile_serialization=False)
torch.save(my_ws, f'{JoJo_folder}{search_image_nu}_my_ws.torch', _use_new_zipfile_serialization=False)
#if jojo_facedetect_output == "jpgs_and_variables":
 #   FACES, device, aligned_faces, my_ws=jojo_cuda_DetectResize(image_folder_nu, search_image_nu)
  #  torch.save(aligned_faces, f'{mugshotGAN_folder}{search_image_nu}_aligned_faces.torch', _use_new_zipfile_serialization=False)
   # torch.save(my_ws, f'{JoJo_folder}{search_image_nu}_my_ws.torch', _use_new_zipfile_serialization=False)
#else:
 #   print(image_folder_nu)
  #  FACES, device, aligned_faces, my_ws=jojo_cuda_DetectResize_nosave(image_folder_nu)
   # torch.save(aligned_faces, f'{mugshotGAN_folder}{search_image_nu}_aligned_faces.torch', _use_new_zipfile_serialization=False)
    #torch.save(my_ws, f'{JoJo_folder}{search_image_nu}_my_ws.torch', _use_new_zipfile_serialization=False)

In [ ]:
#@title Crop new faces: { form-width: "120px", display-mode: "form" }
#RealESRGAN_result_folder_nu=RESULTS+search_image_nu+'_ESRGAN/'
RealESRGAN_result_folder_nu=RESULTS+'{}_FACES/'.format(search_image_nu)

#@markdown Size should match training size:
width_and_height = 1024 #@param ["256", "512", "1024"] {type:"raw"}

import cv2
import dlib
import sys
import os

def FACE_CROPPER_nuface (folder, width=width_and_height, height=width_and_height):
    croppedFACES_nu=RESULTS+'{}_crop/'.format(search_image_nu)
    foldermaker(croppedFACES_nu)
    
    A_folder_nu=croppedFACES_nu+'A/'
    B_folder_nu=croppedFACES_nu+'B/'
    foldermaker(A_folder_nu)
    foldermaker(B_folder_nu)

    A_train_nu=A_folder_nu+'train/'
    B_train_nu=B_folder_nu+'train/'
    foldermaker(A_train_nu)
    foldermaker(B_train_nu)

    A_val_nu=A_folder_nu+'val/'
    B_val_nu=B_folder_nu+'val/'
    foldermaker(A_val_nu)
    foldermaker(B_val_nu)
    
    A_test_nu=A_folder_nu+'test/'
    B_test_nu=B_folder_nu+'test/'
    foldermaker(A_test_nu)
    foldermaker(B_test_nu)
    
    dim=(width, height)

#model ================================================================
    detector=dlib.get_frontal_face_detector()
    model=dlib.shape_predictor(JoJo_folder+'models/dlibshape_predictor_68_face_landmarks.dat')

#source=================================================================
    FACES, frameCOUNT=list_loader(folder)

#loop trhu folder ======================================================
    for f in range(0,frameCOUNT):
        print(f)

#string path ===========================================================
        imash=folder+FACES[f]
        img_extensions=('.jpg','.png')

#only JPGS AND PNGS  ===================================================
        if imash.endswith(img_extensions):
            try:

#DLIB LOAD==============================================================
                IMG=dlib.load_rgb_image(imash)   
#CV LOAD  ==============================================================
                #IMG = cv2.imread(imash)[:,:,::-1]
            
#find face =============================================================
                FACE=detector(IMG, 1)
            
#Pass2landmark =========================================================
                landmark_array=[]
                for k, d, in enumerate(FACE):
                    landmarks=model(IMG, d)

#select only landmarks 0-27 to cutout face=============================
                    for n in range(0,27):
                        x=landmarks.part(n).x
                        y=landmarks.part(n).y
                        landmark_array.append((x, y))
                        #cv2.circle(IMG, (x,y), 2, (255, 255, 0), -1)
            
 #detect if face is frontal ("manually")
                frontson1=landmark_array[0][1] 
                frontson2=landmark_array[17][1]

                cutout_guide=[]

#outline path from landmarks 0-27 ===================================
                for i in range (15, -1, -1):
                    from_coordinate=landmark_array[i+1]
                    to_coordinate=landmark_array[i]
                    cutout_guide.append(from_coordinate)
            
                from_coordinate=landmark_array[0]
                to_coordinate=landmark_array[17]
                cutout_guide.append(from_coordinate)

                for i in range (17, 20):
                    from_coordinate=landmark_array[i]
                    to_coordinate=landmark_array[i+1]
                    cutout_guide.append(from_coordinate)
            
                from_coordinate=landmark_array[19]
                to_coordinate=landmark_array[24]
                cutout_guide.append(from_coordinate)

                for i in range (24, 26):
                    from_coordinate=landmark_array[i]
                    to_coordinate=landmark_array[i+1]
                    cutout_guide.append(from_coordinate)
                
                from_coordinate=landmark_array[26]
                to_coordinate=landmark_array[16]
                cutout_guide.append(from_coordinate)
                cutout_guide.append(to_coordinate)

#outline face===============================================================
                for i in range(0, len(cutout_guide)-1):
                    from_coordinates=cutout_guide[i]
                    to_coordinates=cutout_guide[i+1]
                    IMG2=cv2.line(IMG, from_coordinate, to_coordinate, (255,255,255), 1)
            
#crop it    ================================================================
                mask=np.zeros((IMG2.shape[0], IMG2.shape[1]))
                mask=cv2.fillConvexPoly(mask, np.array(cutout_guide), 1)
                mask=mask.astype(np.bool_)
                cropped=np.zeros_like(IMG2)
                cropped[mask]=IMG2[mask]

#convert to rgb to avoid blue image from BGR ===============================
                IMG=cv2.cvtColor(IMG, cv2.COLOR_BGR2RGB)
                cropped=cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)
                
#jpg cropped  ==============================================================
                IMG=cv2.resize(IMG, dim, interpolation = cv2.INTER_AREA)
                cropped=cv2.resize(cropped, dim, interpolation = cv2.INTER_AREA)
                data_division=int(frameCOUNT*0.92)
                if f <= data_division:
                    cv2.imwrite(B_test_nu+'{}_crop{}.jpg'.format(search_image_nu, f), cropped)
                    cv2.imwrite(A_test_nu+'A{}.jpg'.format(f), IMG)
                elif f >= data_division and f <= (int(data_division*1.15)):
                    cv2.imwrite(A_val_nu+'A{}.jpg'.format(f), IMG)
                    cv2.imwrite(B_val_nu+'B{}.jpg'.format(f), cropped)
                else:
                    cv2.imwrite(A_train_nu+'A{}.jpg'.format(f), IMG)
                    cv2.imwrite(B_train_nu+'B{}.jpg'.format(f), cropped)
                f+=1
            except Exception:
                pass
    return croppedFACES_nu

croppedFACES_nu=FACE_CROPPER_nuface(RealESRGAN_result_folder_nu)

In [ ]:
#@title Format DATA to test pix2pix GAN.
#@markdown (A, B, and AB with corresponding "test," "train" and "val" folder.)
#croppedFACES=RESULTS+'{}_crop/'.format(search_image)

A_folder_nu=croppedFACES_nu+'A/'
B_folder_nu=croppedFACES_nu+'B/'
AB_folder_nu=croppedFACES_nu+'AB/'
foldermaker(AB_folder_nu)

def combine_AB(A, B, path_AB):
    Aa, frameCOUNT=list_loader(A)
    Bb, frameCOUNT=list_loader(B)
    foldermaker(path_AB)
    print(frameCOUNT)

    for f in range (0, frameCOUNT):
        print(f)
        path_A=A+Aa[f]
        path_B=B+Bb[f]
    
        im_A = cv2.imread(path_A) # python2: cv2.CV_LOAD_IMAGE_COLOR; python3: cv2.IMREAD_COLOR
        im_B = cv2.imread(path_B) # python2: cv2.CV_LOAD_IMAGE_COLOR; python3: cv2.IMREAD_COLOR
        im_AB = np.concatenate([im_A, im_B],1)

        cv2.imwrite(path_AB+'{}_AB{}.jpg'.format(search_image_nu,f), im_AB)
        f+=1
combine_AB(A_folder_nu+'train/', B_folder_nu+'train/', AB_folder_nu+'train/')
combine_AB(A_folder_nu+'test/', B_folder_nu+'test/', AB_folder_nu+'test/')
combine_AB(A_folder_nu+'val/', B_folder_nu+'val/', AB_folder_nu+'val/')

#!python datasets/combine_A_and_B.py --fold_A {A_folder} --fold_B {B_folder} 

In [ ]:
#@title load pix2pix 
#@markdown optionally download git if its not already in work_folder:
download_pix2pix = False #@param {type:"boolean"}
if download_pix2pix==True:
    !git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix {pix2pix_folder}
%cd {pix2pix_folder}
!pip install -r requirements.txt

In [ ]:
#@title Test pix2pix on new cropped faces.
#@markdown (Setting VARS again for autonomy, also VERY helpful if crash.)

#@markdown Use default mugshotGAN_4 checkpoint or use your version, just keep checkpoint name same as training. 

#@markdown (Make sure your checkpoint version is in pix2pix 'checkpoints' folder, if choosing mugshotGAN_4 it will DOWNLOAD from mint.tokyo & be there automatically.)
model_checkpoint = "mugshotGAN_4" #@param ["mugshotGAN_4"] {allow-input: true}
if model_checkpoint == "mugshotGAN_4":
    %cd {pix2pix_folder+'checkpoints/'}
    foldermaker(pix2pix_folder+'checkpoints/mugshotGAN_4/')
    !wget -nc https://mint.tokyo/MUGSHOTGAN/MUGSHOTGAN_4.zip
    !sudo unzip -n MUGSHOTGAN_4.zip -d {pix2pix_folder+'checkpoints/mugshotGAN_4'}
    %cd
    %cd {pix2pix_folder}
#@markdown Keep same GAN direction too:
model_direction = "BtoA" #@param ["AtoB", "BtoA"]

#@markdown (if you are using the mugshotGAN_4 checkpoint, direction is BtoA)
croppedFACES_nu=RESULTS+'{}_crop/'.format(search_image_nu)
PIX2PIX_testdata, pix2pixtest_length=list_loader(croppedFACES_nu+'AB/test')
print(croppedFACES_nu)
pix2pix_results=RESULTS+'{}_PIX2PIX/'.format(search_image_nu)
!python test.py --dataroot {croppedFACES_nu+'AB/'} --name {model_checkpoint} --model pix2pix --direction {model_direction} --num_test {pix2pixtest_length*4} --num_threads 8 --no_dropout --serial_batches --results_dir {pix2pix_results}

# Optionally, create JoJoGAN styles with your own images.

In [ ]:
#@title Upload your own portraits for CUSTOM STYLE or skip these 2 steps for default MUGSHOT.
#@markdown

#@markdown ENABLE COOKIES 2 UPLOAD FROM BROWSER!

#@markdown (you may manually place images directly in drive folder, just run this cell empty to create folder and format images with cell below)
%cd
JoJoSTYLES=mugshotGAN_folder+'jojo_styles/'
foldermaker(JoJoSTYLES) 
upload_styles = False #@param {type:"boolean"}
if upload_styles == True:
    %cd {JoJoSTYLES}
    from google.colab import files
    uploaded = files.upload()
    for fn in uploaded.keys():
        print('User uploaded file "{name}" with length {length} bytes'.format(
            name=fn, length=len(uploaded[fn])))
#copy_style_boi = False #@param {type:"boolean"}
#if copy_style_boi==True:    
 #   !rsync -a {JoJoSTYLES} {JoJo_folder+'style_images'}

In [ ]:
#@title Convert STYLE imgs to .jpg & send to JoJo style_images folder: { form-width: "200px", display-mode: "form" }
style_images_origin = JoJoSTYLES #@param ["JoJoSTYLES", "image_folder", "image_folder_nu"] {type:"raw", allow-input: true}
def png_converter (folder, output):
    from PIL import Image
    IMG, image_count=list_loader(folder)
    img_ext=('.jpg', '.png', '.PNG', '.JPG')
    for f in range (0, image_count):
        img=folder+IMG[f]
        if img.endswith(img_ext):
            #jpg_boi=Image.open(img)
            jpg_superior=cv2.imread(img)
            #rgb_boi=jpg_boi.convert('RGBX')
            nu_img_name=strip_path_extension(IMG[f])
            #rgb_boi.save(output+'{}.jpg'.format(nu_img_name), quality=110, color=32)
            cv2.imwrite(output+'{}.jpg'.format(nu_img_name), jpg_superior, [cv2.IMWRITE_JPEG_QUALITY, 100])
            print(nu_img_name+'.jpg saved in JoJo/style_images/')
            f+=1
png_converter(style_images_origin, JoJo_folder+'style_images/')

# generate final MUGSHOTS

In [ ]:
#@title Transfer PIX2PIX results into one MEGA folder. { display-mode: "form" }
if search_image_nu != None:
    mega_image_folder=RESULTS+'{}_mega_folder/'.format(search_image_nu)
else:
    mega_image_folder=RESULTS+'{}_mega_folder/'.format(search_image)
foldermaker(mega_image_folder)
#@markdown Enter folder path:
pix2pix_results=RESULTS+'{}_PIX2PIX/'.format(search_image_nu)
face_folder=pix2pix_results+'/mugshotGAN_4/test_latest/images/'#@param
#!rsync -a {face_folder} {mega_image_folder}
!rsync -am --include '*fake_B.png' --exclude '*real_A.png' --include '*real_B.png' {face_folder} {mega_image_folder} 

In [ ]:
#@title Run Real-ESRGAN to upscale the generated image DATASET.
# Clone Real-ESRGAN and enter the Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git {RealESRGAN_folder}
%cd {RealESRGAN_folder}
# Set up the environment
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop

# Download the pre-trained model
!wget -nc https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models

import shutil
JOJOfakes=RESULTS+'{}_FAKES/'.format(search_image_nu)
os.makedirs('temp_inputs', exist_ok=True) 
FACES=RESULTS+'{}_FACES/'.format(search_image_nu)
RealESRGAN_load_folder = FACES
RealESERGAN_upload_folder=RealESRGAN_folder+'temp_inputs/'
#!rsync -a {RealESRGAN_load_folder} {RealESRGAN_upload_folder}
JOJO_results = False 
if search_image_nu != None and JOJO_results == True:
    RealESRGAN_result_folder=RESULTS+search_image_nu+'_jojoFAKE_ESRGAN/'
elif search_image_nu != None:
    fake_folder=RESULTS+search_image_nu+'_FAKE/'
    RealESRGAN_result_folder = RESULTS+search_image_nu+'_FAKE_ESRGAN/'    
else:
    RealESRGAN_result_folder = RESULTS+search_image+'_ESRGAN/'
foldermaker(RealESRGAN_result_folder)
realERSGAN_input = mega_image_folder #@param ["mega_image_folder", "FACES", "image_folder", "None", "face_folder", "fake_folder", "JOJOfakes"] {type:"raw"}
!python inference_realesrgan.py -n RealESRGAN_x4plus --input {realERSGAN_input} --outscale 4 --half --face_enhance --output {RealESRGAN_result_folder}

In [ ]:
#@title Download and/or load JoJoGAN models.
#@markdown SKIP PYDRIVE DOWNLOAD if models in folder.
#
#@markdown Defaults to downloaded models. SMASH THAT GO BUTTON!
%cd {JoJo_folder}

download_with_pydrive = True #@param {type:"boolean"}    
device = 'cuda' #@param ['cuda', 'cpu']

!wget -nc http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -dk shape_predictor_68_face_landmarks.dat.bz2
!mv shape_predictor_68_face_landmarks.dat models/dlibshape_predictor_68_face_landmarks.dat
#!cp {JoJoMODELS}shape_predictor_68_face_landmarks.dat models/dlibshape_predictor_68_face_landmarks.dat
#!cp {JoJoMODELS}stylegan2-ffhq-config-f.pt models/stylegan2-ffhq-config-f.pt
#!cp {JoJoMODELS}e4e_ffhq_encode.pt models/e4e_ffhq_encode.pt
%matplotlib inline


drive_ids = {
    "stylegan2-ffhq-config-f.pt": "1Yr7KuD959btpmcKGAUsbAk5rPjX2MytK",
    "e4e_ffhq_encode.pt": "1o6ijA3PkcewZvwJJ73dJ0fxhndn0nnh7",
    "restyle_psp_ffhq_encode.pt": "1nbxCIVw9H3YnQsoIPykNEFwWJnHVHlVd",
    "arcane_caitlyn.pt": "1gOsDTiTPcENiFOrhmkkxJcTURykW1dRc",
    "arcane_caitlyn_preserve_color.pt": "1cUTyjU-q98P75a8THCaO545RTwpVV-aH",
    "arcane_jinx_preserve_color.pt": "1jElwHxaYPod5Itdy18izJk49K1nl4ney",
    "arcane_jinx.pt": "1quQ8vPjYpUiXM4k1_KIwP4EccOefPpG_",
    "arcane_multi_preserve_color.pt": "1enJgrC08NpWpx2XGBmLt1laimjpGCyfl",
    "arcane_multi.pt": "15V9s09sgaw-zhKp116VHigf5FowAy43f",
    "sketch_multi.pt": "1GdaeHGBGjBAFsWipTL0y-ssUiAqk8AxD",
    "disney.pt": "1zbE2upakFUAx8ximYnLofFwfT8MilqJA",
    "disney_preserve_color.pt": "1Bnh02DjfvN_Wm8c4JdOiNV4q9J7Z_tsi",
    "jojo.pt": "13cR2xjIBj8Ga5jMO7gtxzIJj2PDsBYK4",
    "jojo_preserve_color.pt": "1ZRwYLRytCEKi__eT2Zxv1IlV6BGVQ_K2",
    "jojo_yasuho.pt": "1grZT3Gz1DLzFoJchAmoj3LoM9ew9ROX_",
    "jojo_yasuho_preserve_color.pt": "1SKBu1h0iRNyeKBnya_3BBmLr4pkPeg_L",
    "art.pt": "1a0QDEHwXQ6hE_FcYEyNMuv5r5UnRQLKT",
}

# from StyelGAN-NADA
class Downloader(object):
    def __init__(self, use_pydrive):
        self.use_pydrive = use_pydrive

        if self.use_pydrive:
            self.authenticate()
        
    def authenticate(self):
        auth.authenticate_user()
        gauth = GoogleAuth()
        gauth.credentials = GoogleCredentials.get_application_default()
        self.drive = GoogleDrive(gauth)
    
    def download_file(self, file_name):
        file_dst = os.path.join('models', file_name)
        file_id = drive_ids[file_name]
        if not os.path.exists(file_dst):
            print(f'Downloading {file_name}')
            if self.use_pydrive:
                downloaded = self.drive.CreateFile({'id':file_id})
                downloaded.FetchMetadata(fetch_all=True)
                downloaded.GetContentFile(file_dst)
            else:
                !gdown --id $file_id -O $file_dst



#downloader = Downloader(download_with_pydrive)

#downloader.download_file('stylegan2-ffhq-config-f.pt')
#downloader.download_file('e4e_ffhq_encode.pt')

latent_dim = 512

# Load original generator
original_generator = Generator(1024, latent_dim, 8, 2).to(device)
ckpt = torch.load('models/stylegan2-ffhq-config-f.pt', map_location=lambda storage, loc: storage)
original_generator.load_state_dict(ckpt["g_ema"], strict=False)
mean_latent = original_generator.mean_latent(10000)

# to be finetuned generator
generator = deepcopy(original_generator)


transform = transforms.Compose(
    [
        transforms.Resize((1024, 1024)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

In [ ]:
#@title Run JoJo face detect, resize and crop TARGET faces. { form-width: "400px", display-mode: "form" }
#@markdown Choose OUTPUT type:
jojo_facedetect_output = "only_variables" #@param ["jpgs_and_variables", "only_variables"]


def jojo_cuda_DetectResize(folder=image_folder_nu, search_image=search_image_nu):

#cuda as device ===================================================================
    device = 'cuda'

 #result folder n variable ========================================================   
    FAKES=RESULTS+search_image_nu+'_FAKES/'
    foldermaker(FAKES)
    aligned_faces=[]
    my_ws=[]

#load folder======================================================================
    filepath, total_files=list_loader(folder)
    total_files1=int(total_files/2)
#loop thru folder=================================================================
    for f in range (0, total_files1):
        img=folder+filepath[f]
        #print(img)

#skip folders n weird formats ===================================================
        img_extensions=('.jpg', '.png')
        if img.endswith(img_extensions):
            try:
                plt.rcParams['figure.dpi'] = 150

# uploaded = files.upload()
# filepath = list(uploaded.keys())[0]
                name = strip_path_extension(img)+'.pt'

# aligns and crops face =====================
                aligned_face= align_face(img)
                aligned_face.save(FACES+'{}_JOJOface_{}.jpg'.format(search_image, f), "JPEG", quality=100, optimize=True, progressive=True)
                my_w = e4e_projection(aligned_face, name, device).unsqueeze(0)
                #print(aligned_face)
                #display_image(aligned_face, title='Aligned face')
                aligned_faces.append(aligned_face)
                my_ws.append(my_w)
                f+=1
            except Exception:
                pass
    
    for f in range (total_files1, total_files):
        img=folder+filepath[f]
        #print(img)

#skip folders n weird formats ===================================================
        img_extensions=('.jpg', '.png')
        if img.endswith(img_extensions):
            try:
                plt.rcParams['figure.dpi'] = 150

# uploaded = files.upload()
# filepath = list(uploaded.keys())[0]
                name = strip_path_extension(img)+'.pt'

# aligns and crops face =====================
                aligned_face= align_face(img)
                aligned_face.save(FACES+'{}_JOJOface_{}.jpg'.format(search_image, f), "JPEG", quality=100, optimize=True, progressive=True)
                my_w = e4e_projection(aligned_face, name, device).unsqueeze(0)
                #print(aligned_face)
                #display_image(aligned_face, title='Aligned face')
                aligned_faces.append(aligned_face)
                my_ws.append(my_w)
                f+=1
            except Exception:
                pass
    return (FAKES, device, aligned_faces, my_ws)


def jojo_cuda_DetectResize_nosave(folder=image_folder_nu):

#cuda as device ===================================================================
    device = 'cuda'

 #result folder n variable ========================================================   
    #FAKES=RESULTS+search_image_nu+'_FAKES/'
    #foldermaker(FAKES)
    aligned_faces=[]
    my_ws=[]

#load folder======================================================================
    filepath, total_files=list_loader(folder)
    total_files1=int(total_files/2)
#loop thru folder=================================================================
    for f in range (0, total_files1):
        img=folder+filepath[f]
        #print(img)

#skip folders n weird formats ===================================================
        img_extensions=('.jpg', '.png')
        if img.endswith(img_extensions):
            try:
                plt.rcParams['figure.dpi'] = 150

# uploaded = files.upload()
# filepath = list(uploaded.keys())[0]
                name = strip_path_extension(img)+'.pt'

# aligns and crops face =====================
                aligned_face= align_face(img)
                #aligned_face.save(FACES+'{}_JOJOface_{}.jpg'.format(search_image, f), "JPEG", quality=100, optimize=True, progressive=True)

                # my_w = restyle_projection(aligned_face, name, device, n_iters=1).unsqueeze(0)
                my_w = e4e_projection(aligned_face, name, device).unsqueeze(0)

                #print(aligned_face)
                #display_image(aligned_face, title='Aligned face')
                my_ws.append(my_w)
                aligned_faces.append(aligned_face)
                f+=1
            except Exception:
                pass

    for f in range (total_files1, total_files):
        img=folder+filepath[f]
        #print(img)

    #skip folders n weird formats ===================================================
        img_extensions=('.jpg', '.png')
        if img.endswith(img_extensions):
            try:
                plt.rcParams['figure.dpi'] = 150

    # uploaded = files.upload()
    # filepath = list(uploaded.keys())[0]
                name = strip_path_extension(img)+'.pt'

    # aligns and crops face =====================
                aligned_face= align_face(img)
                #aligned_face.save(FACES+'{}_JOJOface_{}.jpg'.format(search_image, f), "JPEG", quality=100, optimize=True, progressive=True)

                # my_w = restyle_projection(aligned_face, name, device, n_iters=1).unsqueeze(0)
                my_w = e4e_projection(aligned_face, name, device).unsqueeze(0)

                #print(aligned_face)
                #display_image(aligned_face, title='Aligned face')
                my_ws.append(my_w)
                aligned_faces.append(aligned_face)
                f+=1
            except Exception:
                pass
    return (device, aligned_faces, my_ws)

#@markdown *This cell autosaves a pytorch checkpoint VARS to reuse in case of crash.

run_after_pix2pix = True #@param {type:"boolean"}
#use_fake_folder= False #@param {type:"boolean"}
mega_image_folder=RESULTS+search_image_nu+'_mega_folder/'
fake_folder=RESULTS+search_image_nu+'_FAKE/'
if run_after_pix2pix == True :
    RealESRGAN_result_folder=RESULTS+search_image_nu+'_FAKE_ESRGAN/'
    image_folder_nu=mega_image_folder
    #RealESRGAN_result_folder
fake_input_folder = RealESRGAN_result_folder #@param ["RealESRGAN_result_folder", "image_folder_nu", "mega_image_folder"] {type:"raw", allow-input: true}
if jojo_facedetect_output == "jpgs_and_variables":
    FAKES, device, aligned_faces, my_ws=jojo_cuda_DetectResize(fake_input_folder, search_image_nu)
    torch.save(aligned_faces, f'{mugshotGAN_folder}{search_image_nu}_aligned_faces.torch', _use_new_zipfile_serialization=False)
    torch.save(my_ws, f'{JoJo_folder}{search_image_nu}_my_ws.torch', _use_new_zipfile_serialization=False)
else:
    print(fake_input_folder)
    device, aligned_faces, my_ws=jojo_cuda_DetectResize_nosave(fake_input_folder)
    torch.save(aligned_faces, f'{mugshotGAN_folder}{search_image_nu}_aligned_faces.torch', _use_new_zipfile_serialization=False)
    torch.save(my_ws, f'{JoJo_folder}{search_image_nu}_my_ws.torch', _use_new_zipfile_serialization=False)

In [ ]:
#@title Run this cell to reload pytorch checkpoints for jojo face variables in case of crash. { form-width: "150px", display-mode: "form" }
#@markdown You can skip if this is first run.
load_checkpoints = True #@param {type:"boolean"}
if load_checkpoints == True:
    my_ws=torch.load(f'{JoJo_folder}{search_image_nu}_my_ws.torch')
    aligned_faces=torch.load(f'{mugshotGAN_folder}{search_image_nu}_aligned_faces.torch')
    FACES=RESULTS+'{}_FACES/'.format(search_image_nu)

In [ ]:
#@title  { form-width: "100px", display-mode: "form" }
#@title  { display-mode: "form" }
#@markdown Type style images names into the field bellow without the directory name. Upload multiple style images to do multi-shot image translation
#@markdown
#@markdown (TICK BOX TO USE CAPI MUGSHOT)
%cd {JoJo_folder}
use_capis_mugshot = True #@param {type:"boolean"}
if use_capis_mugshot == True:
    #%cd {JoJo_folder+'style_images/'}
    !wget -nc https://mint.tokyo/MUGSHOTGAN/yop.png -P {JoJo_folder+'style_images/'}
    print('MUGSHOT image ''yop.png'' is ready to use.')
    #%cd {JoJo_folder}
names = ["yop.png"] #@param {type:"raw"}

targets = []
latents = []

for name in names:
    style_path = os.path.join('style_images', name)
    assert os.path.exists(style_path), f"{style_path} does not exist!"

    name = strip_path_extension(name)

    # crop and align the face
    style_aligned_path = os.path.join('style_images_aligned', f'{name}.png')
    if not os.path.exists(style_aligned_path):
        style_aligned = align_face(style_path)
        style_aligned.save(style_aligned_path)
    else:
        style_aligned = Image.open(style_aligned_path).convert('RGB')
    #style_path = os.path.join('{}style_images/'.format(JoJo_folder), name)
    #style_fix = Image.open(style_path).convert('RGB')
    #style_fix.save(style_path)

    #assert os.path.exists(style_path), f"{style_path} does not exist!"

    #name = strip_path_extension(name)

    # crop and align the face
    #style_aligned_path = os.path.join(JoJo_folder+'style_images_aligned', f'{name}.png')
    #if not os.path.exists(style_aligned_path) and style_path.endswith('.png'):
     #   style_aligned = Image.open(style_path).convert('RGB')
      #  style_aligned.save(style_aligned_path)
    #elif not os.path.exists(style_aligned_path):
     #   style_aligned = align_face(style_path)
      #  style_aligned.save(style_aligned_path)
    #else:
     #   style_aligned = Image.open(style_aligned_path).convert('RGB')

    # GAN invert
    style_code_path = os.path.join('inversion_codes', f'{name}.pt')
    if not os.path.exists(style_code_path):
        latent = e4e_projection(style_aligned, style_code_path, device)
    else:
        latent = torch.load(style_code_path)['latent']

    targets.append(transform(style_aligned).to(device))
    latents.append(latent.to(device))

targets = torch.stack(targets, 0)
latents = torch.stack(latents, 0)

target_im = utils.make_grid(targets, normalize=True, range=(-1, 1))
#display_image(target_im, title='Style References')

In [ ]:
#@title Finetune StyleGAN
#@markdown alpha controls the strength of the style
alpha =  0.9 #@param {type:"slider", min:0, max:1, step:0.1}
alpha = 1-alpha

#@markdown Tries to preserve color of original image by limiting family of allowable transformations. Set to false if you want to transfer color from reference image. This also leads to heavier stylization
preserve_color = True #@param{type:"boolean"}
#@markdown Number of finetuning steps. Different style reference may require different iterations. Try 200~500 iterations.
num_iter =  700#@param {type:"number"}
#@markdown Log training on wandb and interval for image logging
use_wandb = False #@param {type:"boolean"}
log_interval = 50 #@param {type:"number"}

latents_temp=latents
targets_temp=targets
target_im_temp=target_im

#print(latents_temp, targets_temp, target_im_temp)

if use_wandb:
    wandb.init(project="JoJoGAN")
    config = wandb.config
    config.num_iter = num_iter
    config.preserve_color = preserve_color
    wandb.log(
    {"Style reference": [wandb.Image(transforms.ToPILImage()(target_im_temp))]},
    step=0)

# load discriminator for perceptual loss
discriminator = Discriminator(1024, 2).eval().to(device)
ckpt = torch.load('models/stylegan2-ffhq-config-f.pt', map_location=lambda storage, loc: storage)
discriminator.load_state_dict(ckpt["d"], strict=False)

# reset generator
del generator
generator = deepcopy(original_generator)

g_optim = optim.Adam(generator.parameters(), lr=2e-3, betas=(0, 0.99))

# Which layers to swap for generating a family of plausible real images -> fake image
if preserve_color:
    id_swap = [9,11,15,16,17]
else:
    id_swap = list(range(7, generator.n_latent))

for idx in tqdm(range(num_iter)):
    mean_w = generator.get_latent(torch.randn([latents_temp.size(0), latent_dim]).to(device)).unsqueeze(1).repeat(1, generator.n_latent, 1)
    in_latent = latents_temp.clone()
    in_latent[:, id_swap] = alpha*latents_temp[:, id_swap] + (1-alpha)*mean_w[:, id_swap]

    img = generator(in_latent, input_is_latent=True)

    with torch.no_grad():
        real_feat = discriminator(targets_temp)
    fake_feat = discriminator(img)

    loss = sum([F.l1_loss(a, b) for a, b in zip(fake_feat, real_feat)])/len(fake_feat)
    
    if use_wandb:
        wandb.log({"loss": loss}, step=idx)
        if idx % log_interval == 0:
            generator.eval()
            my_sample = generator(my_w, input_is_latent=True)
            generator.train()
            my_sample = transforms.ToPILImage()(utils.make_grid(my_sample, normalize=True, range=(-1, 1)))
            wandb.log(
            {"Current stylization": [wandb.Image(my_sample)]},
            step=idx)

    g_optim.zero_grad()
    loss.backward()
    g_optim.step()

In [ ]:
from torchvision.utils import save_image

In [ ]:
#@title Generate results { form-width: "100px", display-mode: "form" }
n_sample =  1#@param {type:"number"}
seed = 3000 #@param {type:"number"}

def JoJoGAN_GENERATOR_one(n_sample=n_sample, seed=seed):

#out folders============================================================
    JoJoGAN_results=RESULTS+'{}_FAKES/'.format(search_image_nu)  
    foldermaker(JoJoGAN_results)

#in folder==============================================================
    #IMG, total_imgs=list_loader(folder)
    total_imgs=int((len(aligned_faces))/2)
    print (total_imgs)

#load ==================================================================
    for f in range (0, total_imgs):
        print(f)
        aligned_face=aligned_faces[f]
        my_w=my_ws[f]
        #try:
        torch.manual_seed(seed)
        with torch.no_grad():
            generator.eval()
            z = torch.randn(n_sample, latent_dim, device=device)

            original_sample = original_generator([z], truncation=0.7, truncation_latent=mean_latent)
            sample = generator([z], truncation=0.7, truncation_latent=mean_latent)

            original_my_sample = original_generator(my_w, input_is_latent=True)
            my_sample = generator(my_w, input_is_latent=True)

        # display reference images
        style_images = []
        for name in names:
            style_path =f'style_images_aligned/{strip_path_extension(name)}.png'
            style_image = transform(Image.open(style_path).convert('RGB'))
            style_images.append(style_image)
            
        #face = transform(aligned_face).to(device).unsqueeze(0)
        #style_images = torch.stack(style_images, 0).to(device)
        #display_image(utils.make_grid(style_images, normalize=True, range=(-1, 1)), title='References')

        #my_output = torch.cat([face, my_sample], 0)
        #display_image(utils.make_grid(my_output, normalize=True, range=(-1, 1)), title='My sample')
        producto=utils.make_grid(my_sample, normalize=True, range=(-1, 1))
        save_image(producto,JoJoGAN_results+'{}_JOJO_{}.png'.format(search_image_nu, f))

        #output = torch.cat([original_sample, sample], 0)
        #display_image(utils.make_grid(output, normalize=True, range=(-1, 1), nrow=n_sample), title='Random samples')
        f+=1
        #except Exception:
         #   pass
    torch.cuda.empty_cache()
JoJoGAN_GENERATOR_one(n_sample, seed)

In [ ]:
#@title Generate results pt 2 for them big bois { form-width: "100px", display-mode: "form" }

def JoJoGAN_GENERATOR_two(n_sample=n_sample, seed=seed):

#out folders============================================================
    JoJoGAN_results=RESULTS+'{}_FAKES/'.format(search_image_nu) 
    foldermaker(JoJoGAN_results)

#in folder==============================================================
    #IMG, total_imgs=list_loader(folder)
    total_imgs=int((len(aligned_faces))/2)
    print (total_imgs)

#load ==================================================================
    for f in range (total_imgs, len(aligned_faces)):
        print(f)
        aligned_face=aligned_faces[f]
        my_w=my_ws[f]
        #try:
        torch.manual_seed(seed)
        with torch.no_grad():
            generator.eval()
            z = torch.randn(n_sample, latent_dim, device=device)

            original_sample = original_generator([z], truncation=0.7, truncation_latent=mean_latent)
            sample = generator([z], truncation=0.7, truncation_latent=mean_latent)

            original_my_sample = original_generator(my_w, input_is_latent=True)
            my_sample = generator(my_w, input_is_latent=True)

        # display reference images
        style_images = []
        for name in names:
            style_path =f'style_images_aligned/{strip_path_extension(name)}.png'
            style_image = transform(Image.open(style_path).convert('RGB'))
            style_images.append(style_image)
            
        #face = transform(aligned_face).to(device).unsqueeze(0)
        #style_images = torch.stack(style_images, 0).to(device)
        #display_image(utils.make_grid(style_images, normalize=True, range=(-1, 1)), title='References')

        #my_output = torch.cat([face, my_sample], 0)
        #display_image(utils.make_grid(my_output, normalize=True, range=(-1, 1)), title='My sample')
        producto=utils.make_grid(my_sample, normalize=True, range=(-1, 1))
        save_image(producto,JoJoGAN_results+'{}_JOJO_{}.png'.format(search_image_nu, f))

        #output = torch.cat([original_sample, sample], 0)
        #display_image(utils.make_grid(output, normalize=True, range=(-1, 1), nrow=n_sample), title='Random samples')
        f+=1
    torch.cuda.empty_cache()
        #except Exception:
         #   pass   
JoJoGAN_GENERATOR_two(n_sample, seed)

In [ ]:
#@title Run Real-ESRGAN to upscale the generated image DATASET.
# Clone Real-ESRGAN and enter the Real-ESRGAN
!git clone https://github.com/xinntao/Real-ESRGAN.git {RealESRGAN_folder}
%cd {RealESRGAN_folder}
# Set up the environment
!pip install basicsr
!pip install facexlib
!pip install gfpgan
!pip install -r requirements.txt
!python setup.py develop

# Download the pre-trained model
!wget -nc https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models

import shutil
JOJOfakes=RESULTS+'{}_FAKES/'.format(search_image_nu)
os.makedirs('temp_inputs', exist_ok=True) 
FACES=RESULTS+'{}_FACES/'.format(search_image_nu)
RealESRGAN_load_folder = FACES
RealESERGAN_upload_folder=RealESRGAN_folder+'temp_inputs/'
#!rsync -a {RealESRGAN_load_folder} {RealESRGAN_upload_folder}
JOJO_results = True #@param {type:"boolean"}
if search_image_nu != None and JOJO_results == True:
    RealESRGAN_result_folder=RESULTS+search_image_nu+'_jojoFAKE_ESRGAN/'
elif search_image_nu != None:
    fake_folder=RESULTS+search_image_nu+'_FAKE/'
    RealESRGAN_result_folder = RESULTS+search_image_nu+'_FAKE_ESRGAN/'    
else:
    RealESRGAN_result_folder = RESULTS+search_image+'_ESRGAN/'
foldermaker(RealESRGAN_result_folder)
realERSGAN_input = JOJOfakes #@param ["mega_image_folder", "FACES", "image_folder", "None", "face_folder", "fake_folder", "JOJOfakes"] {type:"raw"}
!python inference_realesrgan.py -n RealESRGAN_x4plus --input {realERSGAN_input} --outscale 4 --half --face_enhance --output {RealESRGAN_result_folder}

WITH CODE FROM THESE AUTHORS:

Pytorch pix2pix by Jun-Yan Zhu @junyanz
https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

JoJoGAN CoLab by Min Jin Chong @ https://github.com/mchong6/JoJoGAN

REAL ESRGAN by Xintao @
https://github.com/xinntao/Real-ESRGAN.git

google-images-download by Hardik Vasa @hardikvasa forked by Joeclinton1 from https://github.com/Joeclinton1/google-images-download.git

Xavier Weber 
https://towardsdatascience.com/deep-learning-based-super-resolution-with-opencv-4fd736678066

http://dlib.net/imaging.html#get_frontal_face_detector

https://learnopencv.com/edge-detection-using-opencv/

@mohanapranes
https://www.geeksforgeeks.org/cropping-faces-from-images-using-opencv-python/

@misc{sefiks14856,
 author ={Serengil, Sefik Ilkin},
 title = { Facial Landmarks for Face Recognition with Dlib },
 howpublished = { https://sefiks.com/2020/11/20/facial-landmarks-for-face-recognition-with-dlib/ },
 year = { 2020 },
 note = "[Online; accessed 2022-03-19]"
}